~~##### user_coordinate_area_20~24에서 사용자별 지역 방문수 확인~~   
##### -> user_coordinate_area_2020,2021,2022,2023,2024 연도별 지역 방문수 카운팅


In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

In [ ]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))

In [ ]:
#컬렉션 정보
years=['2020', '2021','2022','2023','2024']
result_collection='user_area_frequency'

In [ ]:
#빈도수 계산
years=['2020', '2021','2022','2023','2024']
result_collection='user_area_frequency'
for year in years:
    cl=db.get_collection('user_coordinate_area_'+year)
    result_collection='user_area_frequency_'+year
    cl.aggregate([
        {
            "$match": {
                "region": { "$ne": float('nan') }  # NaN 값 제외
            }
        },
        {
            "$group": {
                "_id": {
                    "uuid": "$uuid",
                    "region": "$region",
                    "destination_area": "$destination_area"
                },
                "count": { "$sum": 1 }  # 빈도수 계산
            }
        },
        {
            "$project": {
                "_id": 0, 
                "uuid": "$_id.uuid",
                "wide-area": "$_id.region",
                "basic-unit": "$_id.destination_area",
                "frequency": "$count"
            }
        },
        {
            "$out": result_collection  # 새로운 컬렉션으로 저장
        }
    ])